In [11]:
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In [13]:
api_key = os.getenv("OPENAI_API_KEY")
llm_config = {"config_list": [{"model": "gpt-4o", "api_key": api_key}]}

## Create Tool to Retrieve Papers from Arvix

In [14]:
import arxiv
from typing import List, Dict, Any
from datetime import  datetime

In [15]:
client = arxiv.Client()

In [16]:
search = arxiv.Search(
    query = "Fine tuning transformer models",
    max_results = 10,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

In [ ]:
for result in client.results(search):
    print(vars(result))

In [18]:
def arxiv_search(query: str, max_results: int = 10) -> List[Dict[str, Any]]:
    """
    Performs a search on Arxiv for papers corresponding to the specified query and returns the top results.

    Args:
        query (str): The search query.
        max_results: The number of search results to return.  Defaults to 10.

    Returns:
        List Dict[str, Any]: A list of articles and corresponding information including title, summary, etc.
    """
    
    results = []

    client = arxiv.Client()
    search = arxiv.Search(
        query = query,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.SubmittedDate
    )

    for result in client.results(search):
        result_dict = vars(result)
        
        # Convert all fields to JSON serializable types
        for key, value in result_dict.items():
            if isinstance(value, datetime):
                result_dict[key] = value.isoformat()
            elif key == "authors":  # Handle Author objects
                result_dict[key] = [str(author) for author in value]  # Convert each Author to string
            elif key == "links":  # Handle Link objects
                result_dict[key] = [str(link) for link in value]  # Convert each Link to string
            elif not isinstance(value, (str, int, float, list, dict, bool, type(None))):
                result_dict[key] = str(value)  # Convert any other non-serializable types to string
        
        results.append(result_dict)
    
    return results



## Create the Agents and Group Chat

In [19]:
from autogen import ConversableAgent, register_function, GroupChat, GroupChatManager

In [ ]:
# Update the FinalReviewer class to properly format the final paper
# Extense classes for the medical evaluation process
class FinalReviewer(ConversableAgent):
    def receive(self, message, sender, request_reply=True, silent=False):
        super().receive(message, sender, request_reply, silent)
        # Check if the final document is satisfactory
        if "satisfactory" in message.get("content", "").lower():
            self.send({"content": "TERMINATE"}, sender)

scientific_research_planner = ConversableAgent(
    name = "ScientificResearchPlanner",
    system_message = """Given a research task, your job is to determine the information required to support the research.  Please make a determination whether Arxiv articles are necessary for substantiating research findings.
    Please provide clear instructions to the scientific research team.  You have the ability to retrieve relevant articles from Arxiv using the provided tool function.  Please limit the search to a maximum of five topics, as no more than five
    search queries to Arxiv are permitted at a time.  Please monitor the task's progress continously and delegate subtasks to other agents as necessary.  If relevant Arxiv articles are not available or do not provide sufficient information to corroborate research
    findings, please suggest alternative strategies or sources.  After each step, please check the current progress and instruct the following steps.  If a step fails for some reason, please try a workaround.
    """,
    description = "You are a planner.  Given a scientific research task, determine what information is required to complete the task.  After each step, please check the progress and instruct the next steps.",
    llm_config = llm_config
)

scientific_reviewer = ConversableAgent(
    name = "ScientificReviewer",
    system_message = """You are a scientific reviewer with expertise in the fields of physics, mathematics, computer science, quantitative biology, quantitative finance, statistics, electrical engineering and systems science, and economics.
    Please review the research findings from a validity, accuracy and significance of research findings standpoint.  Please provide concise, relevant and specific suggestions for improvement and consideration.
    """,
    llm_config = llm_config

)

scientific_researcher = ConversableAgent(
    name = "ScientificResearcher",
    system_message = """You are a scientific researcher responsible for drafting a scientific manuscript that will detail your research findings.  Please ensure that the content of the scientific manuscript is valid and encapsulates the significant findings from your research.
    Please utilize the arxiv_search function for literature, and ensure that you cite at least one relevant source to substantiate your research findings.
    """,
    llm_config = llm_config
)

scientific_editor = ConversableAgent(
    name = "ScientificEditor",
    system_message = """You are a scientific editor.  You are responsible for editing the content to ensure that the grammar and diction are correct.  You should also ensure that the scientific terminology referenced in the paper is relevant and accurate.  Please ensure that the paper is coherent.""",
    llm_config = llm_config
)

final_reviewer = FinalReviewer(
    name = "FinalReviewer",
    system_message = """You are the final reviewer, responsible for consolidating and reviewing feedback from the scientific reviewer.  You are responsible for making the ultimate decision on the content's readiness for publication.
    Please write TERMINATE if the document meets the required criteria and is ready to be published.
    """
)

# user proxy for executing tool calls
user_proxy = ConversableAgent(
    name = "UserProxy",
    llm_config = False,
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", ""),
    human_input_mode = "NEVER"
)

# register the arxiv_search tool for the appropriate agents
for caller in [scientific_researcher, scientific_research_planner]:
    register_function(
        arxiv_search,
        caller = caller,
        executor = user_proxy,
        name = "arxiv_search",
        description = "Retrives relevant articles from the Arxiv archive based on a search query."
    )


scientific_groupchat = GroupChat(
    agents = [
        scientific_reviewer, scientific_editor, scientific_research_planner, scientific_researcher, final_reviewer, user_proxy
    ],
    messages = [],
    max_round = 3,
    allowed_or_disallowed_speaker_transitions = {
        scientific_reviewer: [final_reviewer, scientific_researcher],
        scientific_researcher: [scientific_editor, scientific_reviewer],
        final_reviewer: [scientific_researcher],  # Final reviewer can terminate or request changes
        scientific_editor: [scientific_reviewer, scientific_researcher], 
        scientific_research_planner: [
            scientific_editor, scientific_researcher, scientific_reviewer, final_reviewer,
        ],
        user_proxy: [
            scientific_reviewer, scientific_editor, scientific_research_planner, scientific_researcher, final_reviewer
        ]
    },
    speaker_transitions_type = "allowed"
)


scientific_manager = GroupChatManager(
    groupchat = scientific_groupchat, llm_config = llm_config
)

task = """
    Create a research paper on novel Named Entity Recognition (NER) methods.  Please discuss how Large Language Models (LLMs) can be leveraged as well as classical Natural Language Processing (NLP) and Machine Learning models.  
    Compare and contrast the results of these different methods.  Include a methodology, results, discussion and conclusion section in the research document.  Please ensure the research findings are significant and accurate.  Please 
    utilize relevant articles from Arxiv to support and substantiate your research findings.
"""

# Update the main chat initiation with stricter termination checking
def is_termination_msg(x):
    content = x.get("content", "")
    if "TERMINATE" in content:
        # Extract and print the final paper
        if "FINAL PAPER:" in content:
            paper_start = content.find("FINAL PAPER:")
            paper_end = content.find("TERMINATE")
            final_paper = content[paper_start:paper_end].strip()
            print("\n" + "="*50)
            print(final_paper)
            print("="*50 + "\n")
        return True
    return False

# Update the chat initiation
scientific_groupchat_result = user_proxy.initiate_chat(
    recipient=scientific_manager,
    message=task,
    is_termination_msg=lambda x: "terminate" in x.get("content", "").lower(),
    max_turns=8  # Reduced to prevent excessive conversation
)
